In [1]:
import pickle
import pandas as pd
import numpy as np
# import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
def binary_decision(dec):
    if dec == 'A':return 1
    else: return 0

In [3]:
with open('featuresDFwCat.pkl', 'rb') as fobj:
    df = pickle.load(fobj, encoding='latin1')
with open('updatedProfiles.pkl', 'rb') as fobj:
    df_raw = pickle.load(fobj, encoding='latin1')
        
df.head()

,school,gpa,act,sat1,test_score,test_type,ap_count,ap_mean,sat2_count,sat2_mean,country,gender,ethnicity,urm,first_generation,ec_count,decision
1,stanford,4.00,33.0,2237.403621,2180,act,1.0,4.456758,1.0,800.000000,US,F,caucasian,0,0,1,0
3,stanford,4.00,35.0,2237.403621,2330,act,2.0,4.456758,3.0,796.666667,US,None,None,0,0,0,1
4,stanford,3.98,34.0,2237.403621,2250,act,4.0,3.750000,0.0,757.780283,US,M,caucasian,0,0,0,1
6,stanford,3.90,32.0,2237.403621,2120,act,6.0,3.333333,0.0,757.780283,US,F,None,1,0,3,1
8,stanford,3.98,36.0,2237.403621,2390,act,7.0,4.285714,3.0,773.333333,US,M,caucasian,0,0,0,0


In [4]:
df['decision'].mean()

0.5662878787878788

In [5]:
df = df.loc[df.act != 1]

In [6]:
score_map = {36: 2390, 35: 2330, 34:2250, 33.35567282:2220, 33:2180, 32:2120, 31:2060, 30:2000, 
             29:1940, 28:1880, 27:1820, 26:1770, 25:1710, 24:1650, 23:1590, 22:1530, 21:1470, 1:600, 0:0, 40:0}
sat_mode = float(df['sat1'].mode())
act_mode = int(df['act'].mode())
gpa_mode = float(df['gpa'].mode())

In [7]:
df_raw2 = df_raw[['gpa','act', 'sat1', 'decision']].copy(deep = True)
df_raw2['decision'] = df_raw2['decision'].apply(binary_decision)
df_raw2['act'] = df_raw2['act'].astype(int)

In [8]:
def get_best_test(row):
    if row.act == 0:
        return pd.Series({'test_type': 'sat', 'test_score': row.sat1_round})
    elif row.sat1 == 0:
        return pd.Series({'test_type': 'act', 'test_score': score_map.get(row.act)})
    else:
#         print(row.act)
        if row.sat1_round > score_map.get(row.act):
            return pd.Series({'test_type': 'sat', 'test_score': row.sat1_round})
        else:
            return pd.Series({'test_type': 'act', 'test_score': score_map.get(row.act)})

In [9]:
def myround(x, base=25):
    return int(base * round(float(x)/base))
df_raw2['sat1_round'] = df_raw2['sat1'].apply(myround)

In [10]:
df_raw3['act'].unique()

NameError: name 'df_raw3' is not defined

In [ ]:
df_raw3 = df_raw2.loc[((df_raw2.gpa != 0) & ((df_raw2.sat1 != 0) | (df_raw2.act != 0)))].copy(deep = True)
# df_raw3
df_raw3[['test_score','test_type']] = df_raw3.apply(get_best_test, axis = 1)

In [ ]:
df_graph = df_raw3
df_graph.head()

In [ ]:
# acc = df2.loc[df.decision == 1]
# rej = df2.loc[df.decision == 0]
# # rewaj = df.loc[df.decision == 1]

In [ ]:
g = df_graph.groupby(by = 'act')['decision'].agg(['mean','count']).reset_index()

plt.scatter(g['act'], g['mean'], s=g['count'])
plt.xlim(20, 36)
plt.xlabel('ACT score')
plt.ylabel('Percent Acceptance')
plt.title('ACT vs. Acceptance')
plt.show()

In [ ]:
g = df_graph.groupby(by = 'sat1_round')['decision'].agg(['mean','count']).reset_index()

plt.scatter(g['sat1_round'], g['mean'], s=g['count'])
plt.xlim(1800, 2400)
plt.ylim(.3, .7)
plt.xlabel('SAT score')
plt.ylabel('Percent Acceptance')
plt.title('SAT vs. Acceptance')
plt.show()

In [ ]:
g = df_graph.groupby(by = ['test_type', 'test_score'])['decision'].agg(['mean','count']).reset_index()
g_act = g.loc[g.test_type == 'act']
g_sat = g.loc[g.test_type == 'sat']

plt.scatter(g_act['test_score'], g_act['mean'], s=g_act['count'], c = 'b', label = 'act')
plt.scatter(g_sat['test_score'], g_sat['mean'], s=g_sat['count'], c = 'r', label = 'sat')

plt.xlim(1800, 2400)
plt.ylim(.3, 1)
plt.xlabel('Stardardized Test Score')
plt.ylabel('Percent Acceptance')
plt.title('Test Score vs. Acceptance')
plt.legend(loc='upper left')
plt.show()

In [ ]:
g = df.groupby(by = ['ec_count'])['decision'].agg(['mean','count']).reset_index()
plt.scatter(g['ec_count'], g['mean'], s=g['count'], c = 'b', label = 'act')

plt.xlim(0, 10)
plt.ylim(.3, 1)
plt.xlabel('ECs')
plt.ylabel('Percent Acceptance')
plt.title('ECs vs. Acceptance')
plt.show()

In [ ]:
def decround(x, base=10):
    x = x * 100
    x = int(base * round(float(x)/base))
    return x/100
df['gpa_round'] = df['gpa'].apply(decround)

In [ ]:
df.head(10)
# df['gpa'].unique()

In [ ]:
g = df.groupby(by = ['gpa_round'])['decision'].agg(['mean','count']).reset_index()
plt.scatter(g['gpa_round'], g['mean'], s=g['count'], c = 'b', label = 'act')

plt.xlim(3, 4.25)
plt.ylim(.3, 1)
plt.xlabel('GPA')
plt.ylabel('Percent Acceptance')
plt.title('GPA vs. Acceptance')
plt.show()

In [ ]:
g = df.groupby(by = ['sat2_count'])['decision'].agg(['mean','count']).reset_index()
plt.scatter(g['sat2_count'], g['mean'], s=g['count'], c = 'b', label = 'act')

plt.xlim(0, 10)
plt.ylim(.3, 1)
plt.xlabel('SAT2 Count')
plt.ylabel('Percent Acceptance')
plt.title('SAT2 Count vs. Acceptance')
plt.show()

In [ ]:
g = df.groupby(by = ['ethnicity'])['decision'].agg(['mean','count']).reset_index()
plt.scatter(g['ethnicity'], g['mean'], s=g['count'], c = 'b', label = 'act')

plt.ylim(.3, 1)
plt.xlabel('Ethnicity')
plt.ylabel('Percent Acceptance')
plt.title('Ethnicity vs. Acceptance')
plt.show()

In [ ]:
df['urmstr'] = df['urm'].astype(str)
g = df.groupby(by = ['urmstr'])['decision'].agg(['mean','count']).reset_index()
plt.scatter(g['urmstr'], g['mean'], s=g['count'], c = 'b', label = 'act')

# plt.xlim(0, 10)
plt.ylim(.3, 1)
plt.xlabel('urm')
plt.ylabel('Percent Acceptance')
plt.title('urm vs. Acceptance')
plt.show()

In [ ]:
df['fg'] = df['first_generation'].astype(str)
g = df.groupby(by = ['fg'])['decision'].agg(['mean','count']).reset_index()
plt.scatter(g['fg'], g['mean'], s=g['count'], c = 'b', label = 'act')

# plt.xlim(0, 10)
plt.ylim(.3, 1)
plt.xlabel('First generation')
plt.ylabel('Percent Acceptance')
plt.title('First_generation vs. Acceptance')
plt.show()

In [ ]:
g = df.groupby(by = ['gender'])['decision'].agg(['mean','count']).reset_index()
plt.scatter(g['gender'], g['mean'], s=g['count'], c = 'b', label = 'act')

plt.ylim(.3, 1)
plt.xlabel('Gender')
plt.ylabel('Percent Acceptance')
plt.title('Gender vs. Acceptance')
plt.show()

In [ ]:
df['rank_round'] = df['rank'].apply(decround)
g = df.groupby(by = ['rank_round'])['decision'].agg(['mean','count']).reset_index()
plt.scatter(g['rank_round'], g['mean'], s=g['count'], c = 'b', label = 'act')

plt.xlim(0, .5)
plt.ylim(.3, 1)
plt.xlabel('Rank')
plt.ylabel('Percent Acceptance')
plt.title('Rank vs. Acceptance')
plt.show()

In [ ]:
g = df.groupby(by = ['rank_round'])['decision'].agg(['mean','count']).reset_index()
plt.scatter(g['rank_round'], g['mean'], s=g['count'], c = 'b', label = 'act')

plt.xlim(0, .5)
plt.ylim(.3, 1)
plt.xlabel('Rank')
plt.ylabel('Percent Acceptance')
plt.title('Rank vs. Acceptance')
plt.show()

In [ ]:
bins = np.linspace(1800, 2400, 12)
plt.hist(acc['sat1'], bins, alpha=0.5, label='acc')
plt.hist(rej['sat1'], bins, alpha=0.5, label='rej')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# plt.figure()
df.hist(column = 'act')
df.hist(column = 'sat1')
df.hist(column = 'gpa')
df.hist(column = 'rank')

In [ ]:
df.plot.scatter(x='ec_count', y='decision')